# Currency Assistant - ML Forecasting Engine Experimentation

This notebook provides a complete, self-contained environment to experiment with the ML forecasting engine for foreign exchange rate prediction. All functions are defined explicitly in this notebook for easy experimentation.

## Key Features
- **Real Data Collection**: Download historical FX data from Yahoo Finance
- **LSTM Model**: Advanced neural network with attention mechanism
- **Uncertainty Quantification**: Monte Carlo dropout for prediction confidence
- **Comprehensive Metrics**: MAE, MSE, RMSE, MAPE, R², and more
- **Interactive Experimentation**: Easy parameter tuning and testing


## Setup and Imports

In [ ]:
import asyncio
import logging
import sys
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import matplotlib.pyplot as plt
import seaborn as sns
import yfinance as yf
import pickle
import time
import os
from datetime import datetime, timedelta
from pathlib import Path
from typing import List, Dict, Optional, Tuple, Any
from dataclasses import dataclass, field
from sklearn.preprocessing import StandardScaler, RobustScaler
from sklearn.model_selection import train_test_split

# Setup plotting
plt.style.use('seaborn-v0_8')
plt.rcParams['figure.figsize'] = (12, 6)

# Configure logging
logging.basicConfig(level=logging.INFO, format='%(levelname)s - %(message)s')
logger = logging.getLogger(__name__)

print("✅ Setup complete! Ready for ML experimentation.")
print(f"📊 PyTorch version: {torch.__version__}")
print(f"🖥️  CUDA available: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"🚀 GPU: {torch.cuda.get_device_name(0)}")

# Create necessary directories
os.makedirs('data/historical', exist_ok=True)
os.makedirs('models', exist_ok=True)

## Configuration Classes

In [ ]:
@dataclass
class ModelConfig:
    """Configuration for LSTM forecasting model."""
    sequence_length: int = 168  # 7 days * 24 hours (weekly patterns)
    prediction_horizon: int = 24  # Predict 24 hours ahead
    input_features: int = 10  # Number of input features
    hidden_size: int = 128  # LSTM hidden size
    num_layers: int = 3  # Number of LSTM layers
    dropout: float = 0.2  # Dropout rate
    attention_heads: int = 8  # Multi-head attention
    learning_rate: float = 0.001
    batch_size: int = 32
    epochs: int = 100

@dataclass
class FeatureConfig:
    """Configuration for feature engineering pipeline."""
    # Technical indicators
    rsi_period: int = 14
    ma_short_period: int = 12
    ma_long_period: int = 26
    bb_period: int = 20
    bb_std_dev: float = 2.0
    
    # Volatility features
    volatility_windows: List[int] = field(default_factory=lambda: [6, 12, 24, 48])
    
    # Lag features
    lag_periods: List[int] = field(default_factory=lambda: [1, 2, 3, 6, 12, 24])
    
    # Time features
    include_time_features: bool = True
    
    # Scaling
    scaler_type: str = "robust"  # "standard" or "robust"

@dataclass
class TrainingConfig:
    """Configuration for model training."""
    model_config: ModelConfig = field(default_factory=ModelConfig)
    feature_config: FeatureConfig = field(default_factory=FeatureConfig)
    max_epochs: int = 50
    batch_size: int = 32
    learning_rate: float = 0.001
    patience: int = 10
    validation_split: float = 0.2
    device: str = "auto"  # "auto", "cuda", or "cpu"

@dataclass
class TrainingResult:
    """Results from model training."""
    train_losses: List[float] = field(default_factory=list)
    val_losses: List[float] = field(default_factory=list)
    val_metrics: Dict[str, float] = field(default_factory=dict)
    training_time: float = 0.0
    best_epoch: int = 0
    converged: bool = False

print("✅ Configuration classes defined")

## Data Collection Functions

In [ ]:
class YFinanceDataCollector:
    """
    Collects historical FX data from Yahoo Finance for ML training.
    
    Yahoo Finance provides reliable historical data for major currency pairs
    with good coverage and reasonable data quality.
    """
    
    def __init__(self, data_dir: str = "data/historical"):
        self.data_dir = Path(data_dir)
        self.data_dir.mkdir(parents=True, exist_ok=True)
        
        # Yahoo Finance FX symbol mapping
        self.fx_symbols = {
            'USD/EUR': 'EURUSD=X',
            'USD/GBP': 'GBPUSD=X', 
            'USD/JPY': 'USDJPY=X',
            'EUR/GBP': 'EURGBP=X',
            'EUR/JPY': 'EURJPY=X',
            'GBP/JPY': 'GBPJPY=X',
            'USD/CHF': 'USDCHF=X',
            'EUR/CHF': 'EURCHF=X',
            'GBP/CHF': 'GBPCHF=X',
            'AUD/USD': 'AUDUSD=X',
            'USD/CAD': 'USDCAD=X',
            'NZD/USD': 'NZDUSD=X'
        }
        
        logger.info(f"Initialized YFinance collector with data directory: {self.data_dir}")
    
    def download_historical_data(
        self,
        currency_pairs: List[str],
        period: str = "2y",
        interval: str = "1h",
        save: bool = True
    ) -> Dict[str, pd.DataFrame]:
        """
        Download historical FX data from Yahoo Finance.
        
        Args:
            currency_pairs: List of currency pairs (e.g., ['USD/EUR', 'USD/GBP'])
            period: Time period ('1d', '5d', '1mo', '3mo', '6mo', '1y', '2y', '5y', '10y', 'ytd', 'max')
            interval: Data interval ('1m', '2m', '5m', '15m', '30m', '60m', '90m', '1h', '1d', '5d', '1wk', '1mo', '3mo')
            save: Whether to save data to disk
            
        Returns:
            Dictionary mapping currency pairs to DataFrames
        """
        logger.info(f"Downloading {len(currency_pairs)} currency pairs for period {period} with {interval} interval")
        
        results = {}
        
        for pair in currency_pairs:
            if pair not in self.fx_symbols:
                logger.warning(f"Currency pair {pair} not supported, skipping")
                continue
            
            yahoo_symbol = self.fx_symbols[pair]
            logger.info(f"Downloading {pair} ({yahoo_symbol})...")
            
            try:
                # Download data from Yahoo Finance
                ticker = yf.Ticker(yahoo_symbol)
                data = ticker.history(
                    period=period,
                    interval=interval,
                    auto_adjust=True,
                    prepost=True
                )
                
                if data.empty:
                    logger.error(f"No data received for {pair}")
                    continue
                
                # Clean and prepare data
                df = self._prepare_dataframe(data, pair)
                results[pair] = df
                
                logger.info(f"Downloaded {len(df)} records for {pair} from {df.index.min()} to {df.index.max()}")
                
                # Save to disk if requested
                if save:
                    self._save_data(df, pair, period, interval)
                    
            except Exception as e:
                logger.error(f"Failed to download data for {pair}: {e}")
                continue
        
        logger.info(f"Downloaded {len(results)} currency pairs successfully")
        return results
    
    def _prepare_dataframe(self, data: pd.DataFrame, currency_pair: str) -> pd.DataFrame:
        """Prepare and clean Yahoo Finance data."""
        df = data.copy()
        
        # Rename columns to match our format
        df = df.rename(columns={
            'Open': 'open',
            'High': 'high', 
            'Low': 'low',
            'Close': 'close',
            'Volume': 'volume'
        })
        
        # Use close price as main rate
        df['rate'] = df['close']
        
        # Add currency pair column
        df['currency_pair'] = currency_pair
        df['provider'] = 'YahooFinance'
        
        # Reset index to make timestamp a column
        df = df.reset_index()
        df = df.rename(columns={'Datetime': 'timestamp'})
        
        # Remove any NaN values
        df = df.dropna()
        
        # Sort by timestamp
        df = df.sort_values('timestamp').reset_index(drop=True)
        
        return df
    
    def _save_data(self, df: pd.DataFrame, currency_pair: str, period: str, interval: str):
        """Save DataFrame to disk."""
        filename = f"{currency_pair.replace('/', '_')}_{period}_{interval}.csv"
        filepath = self.data_dir / filename
        
        df.to_csv(filepath, index=False)
        logger.info(f"Saved {len(df)} records to {filepath}")
        
        # Also save as pickle for faster loading
        pickle_path = filepath.with_suffix('.pkl')
        df.to_pickle(pickle_path)
    
    def load_historical_data(
        self,
        currency_pair: str,
        period: str = "2y", 
        interval: str = "1h"
    ) -> Optional[pd.DataFrame]:
        """Load previously downloaded data from disk."""
        filename = f"{currency_pair.replace('/', '_')}_{period}_{interval}.pkl"
        filepath = self.data_dir / filename
        
        if not filepath.exists():
            logger.warning(f"No saved data found for {currency_pair} at {filepath}")
            return None
        
        try:
            df = pd.read_pickle(filepath)
            logger.info(f"Loaded {len(df)} records for {currency_pair} from {filepath}")
            return df
        except Exception as e:
            logger.error(f"Failed to load data for {currency_pair}: {e}")
            return None
    
    def get_available_data(self) -> List[Dict]:
        """Get list of available downloaded data files."""
        available = []
        
        for file_path in self.data_dir.glob("*.pkl"):
            try:
                parts = file_path.stem.split('_')
                if len(parts) >= 3:
                    currency_pair = parts[0] + '/' + parts[1]
                    period = parts[2]
                    interval = parts[3] if len(parts) > 3 else '1h'
                    
                    # Get file info
                    df = pd.read_pickle(file_path)
                    
                    available.append({
                        'currency_pair': currency_pair,
                        'period': period,
                        'interval': interval,
                        'records': len(df),
                        'start_date': df['timestamp'].min(),
                        'end_date': df['timestamp'].max(),
                        'file_path': str(file_path)
                    })
            except Exception as e:
                logger.warning(f"Could not read file {file_path}: {e}")
        
        return available
    
    def get_data_summary(self, df: pd.DataFrame) -> Dict:
        """Get summary statistics for the dataset."""
        if df.empty:
            return {"error": "Empty dataset"}
        
        summary = {
            "total_records": len(df),
            "currency_pairs": df['currency_pair'].nunique(),
            "pairs_list": sorted(df['currency_pair'].unique().tolist()),
            "date_range": {
                "start": df['timestamp'].min(),
                "end": df['timestamp'].max(),
                "days": (df['timestamp'].max() - df['timestamp'].min()).days
            },
            "rate_statistics": {
                "mean": df['rate'].mean(),
                "std": df['rate'].std(),
                "min": df['rate'].min(),
                "max": df['rate'].max(),
                "median": df['rate'].median()
            },
            "missing_values": df.isnull().sum().sum(),
            "data_quality": {
                "complete_days": len(df.groupby(df['timestamp'].dt.date)),
                "avg_records_per_day": len(df) / max(1, (df['timestamp'].max() - df['timestamp'].min()).days)
            }
        }
        
        return summary

print("✅ Data collection functions defined")

## Feature Engineering Functions

In [ ]:
class FeatureEngineering:
    """
    Feature engineering pipeline for FX time series data.
    
    Transforms raw exchange rate data into ML-ready features including:
    - Technical indicators (RSI, Moving Averages, Bollinger Bands)
    - Time-based features (hour, day of week, etc.)
    - Lag variables and returns
    - Volatility measures
    """
    
    def __init__(self, config: FeatureConfig):
        self.config = config
        self.scaler = None
        self.feature_names: List[str] = []
        self.is_fitted = False
    
    def create_technical_indicators(self, df: pd.DataFrame) -> pd.DataFrame:
        """Create technical indicators from OHLC data."""
        result = df.copy()
        
        # RSI (Relative Strength Index)
        result['rsi'] = self._calculate_rsi(result['rate'], self.config.rsi_period)
        
        # Moving averages
        result['ma_short'] = result['rate'].rolling(window=self.config.ma_short_period).mean()
        result['ma_long'] = result['rate'].rolling(window=self.config.ma_long_period).mean()
        result['ma_ratio'] = result['ma_short'] / result['ma_long']
        
        # Bollinger Bands
        bb_mean = result['rate'].rolling(window=self.config.bb_period).mean()
        bb_std = result['rate'].rolling(window=self.config.bb_period).std()
        result['bb_upper'] = bb_mean + (bb_std * self.config.bb_std_dev)
        result['bb_lower'] = bb_mean - (bb_std * self.config.bb_std_dev)
        result['bb_position'] = (result['rate'] - result['bb_lower']) / (result['bb_upper'] - result['bb_lower'])
        
        # Price momentum
        result['momentum_24h'] = result['rate'].pct_change(periods=24)
        result['momentum_12h'] = result['rate'].pct_change(periods=12)
        result['momentum_6h'] = result['rate'].pct_change(periods=6)
        
        return result
    
    def create_volatility_features(self, df: pd.DataFrame) -> pd.DataFrame:
        """Create volatility-based features."""
        result = df.copy()
        
        # Returns for volatility calculation
        result['returns'] = result['rate'].pct_change()
        
        # Rolling volatilities
        for window in self.config.volatility_windows:
            col_name = f'volatility_{window}h'
            result[col_name] = result['returns'].rolling(window=window).std()
        
        # Volatility of volatility (second-order)
        if 'volatility_24h' in result.columns:
            result['vol_of_vol'] = result['volatility_24h'].rolling(window=24).std()
        
        # Realized vs implied volatility proxy
        if 'volatility_6h' in result.columns and 'volatility_24h' in result.columns:
            result['vol_ratio'] = result['volatility_6h'] / (result['volatility_24h'] + 1e-8)
        
        return result
    
    def create_time_features(self, df: pd.DataFrame) -> pd.DataFrame:
        """Create time-based cyclical features."""
        if not self.config.include_time_features:
            return df
        
        result = df.copy()
        
        # Ensure timestamp is datetime
        if not pd.api.types.is_datetime64_any_dtype(result['timestamp']):
            result['timestamp'] = pd.to_datetime(result['timestamp'])
        
        # Hour of day (cyclical encoding)
        result['hour_sin'] = np.sin(2 * np.pi * result['timestamp'].dt.hour / 24)
        result['hour_cos'] = np.cos(2 * np.pi * result['timestamp'].dt.hour / 24)
        
        # Day of week (cyclical encoding)
        result['dow_sin'] = np.sin(2 * np.pi * result['timestamp'].dt.dayofweek / 7)
        result['dow_cos'] = np.cos(2 * np.pi * result['timestamp'].dt.dayofweek / 7)
        
        # Month of year (for seasonal patterns)
        result['month_sin'] = np.sin(2 * np.pi * result['timestamp'].dt.month / 12)
        result['month_cos'] = np.cos(2 * np.pi * result['timestamp'].dt.month / 12)
        
        # Market session indicators
        result['is_market_hours'] = self._is_market_hours(result['timestamp'])
        result['is_weekend'] = (result['timestamp'].dt.dayofweek >= 5).astype(int)
        
        return result
    
    def create_lag_features(self, df: pd.DataFrame) -> pd.DataFrame:
        """Create lagged features and returns."""
        result = df.copy()
        
        # Lagged rates
        for lag in self.config.lag_periods:
            result[f'rate_lag_{lag}'] = result['rate'].shift(lag)
        
        # Lagged returns
        returns = result['rate'].pct_change()
        for lag in self.config.lag_periods:
            result[f'returns_lag_{lag}'] = returns.shift(lag)
        
        # Lagged technical indicators
        if 'rsi' in result.columns:
            for lag in [1, 6, 12]:
                result[f'rsi_lag_{lag}'] = result['rsi'].shift(lag)
        
        return result
    
    def prepare_features(self, df: pd.DataFrame) -> pd.DataFrame:
        """Complete feature engineering pipeline."""
        logger.info("Starting feature engineering pipeline")
        
        # Ensure data is sorted by timestamp
        df = df.sort_values('timestamp').reset_index(drop=True)
        
        # Create all feature types
        df = self.create_technical_indicators(df)
        df = self.create_volatility_features(df)
        df = self.create_time_features(df)
        df = self.create_lag_features(df)
        
        # Remove rows with NaN values (from indicators and lags)
        initial_rows = len(df)
        df = df.dropna()
        final_rows = len(df)
        
        logger.info(f"Feature engineering complete: {initial_rows} -> {final_rows} rows")
        
        return df
    
    def fit_scaler(self, df: pd.DataFrame, feature_columns: List[str]) -> None:
        """Fit the scaler on training data."""
        if self.config.scaler_type == "standard":
            self.scaler = StandardScaler()
        else:
            self.scaler = RobustScaler()
        
        self.scaler.fit(df[feature_columns])
        self.feature_names = feature_columns
        self.is_fitted = True
        
        logger.info(f"Fitted {self.config.scaler_type} scaler on {len(feature_columns)} features")
    
    def transform_features(self, df: pd.DataFrame) -> np.ndarray:
        """Transform features using fitted scaler."""
        if not self.is_fitted:
            raise ValueError("Scaler not fitted. Call fit_scaler() first.")
        
        return self.scaler.transform(df[self.feature_names])
    
    def fit_transform_features(self, df: pd.DataFrame, feature_columns: List[str]) -> np.ndarray:
        """Fit scaler and transform features in one step."""
        self.fit_scaler(df, feature_columns)
        return self.transform_features(df)
    
    def get_feature_columns(self, df: pd.DataFrame) -> List[str]:
        """Get list of feature columns (excluding target and metadata)."""
        exclude_cols = ['timestamp', 'rate', 'currency_pair', 'provider', 'bid', 'ask', 'volume', 'open', 'high', 'low', 'close']
        feature_cols = [col for col in df.columns if col not in exclude_cols]
        return feature_cols
    
    def _calculate_rsi(self, prices: pd.Series, period: int = 14) -> pd.Series:
        """Calculate Relative Strength Index."""
        delta = prices.diff()
        gain = delta.where(delta > 0, 0).rolling(window=period).mean()
        loss = (-delta.where(delta < 0, 0)).rolling(window=period).mean()
        rs = gain / (loss + 1e-8)
        rsi = 100 - (100 / (1 + rs))
        return rsi
    
    def _is_market_hours(self, timestamps: pd.Series) -> pd.Series:
        """Determine if timestamp falls within major market hours."""
        is_weekday = timestamps.dt.dayofweek < 5
        is_business_hour = (timestamps.dt.hour >= 8) & (timestamps.dt.hour <= 18)
        return (is_weekday & is_business_hour).astype(int)


class SequenceGenerator:
    """Generate sequences for LSTM training from time series data."""
    
    def __init__(self, sequence_length: int = 168, prediction_horizon: int = 24):
        self.sequence_length = sequence_length
        self.prediction_horizon = prediction_horizon
    
    def create_sequences(
        self, 
        features: np.ndarray, 
        targets: np.ndarray,
        stride: int = 1
    ) -> Tuple[np.ndarray, np.ndarray]:
        """Create input-output sequences for LSTM training."""
        n_samples, n_features = features.shape
        
        # Calculate number of sequences we can create
        n_sequences = (n_samples - self.sequence_length - self.prediction_horizon + 1) // stride
        
        X = np.zeros((n_sequences, self.sequence_length, n_features))
        y = np.zeros((n_sequences, self.prediction_horizon))
        
        for i in range(n_sequences):
            start_idx = i * stride
            end_idx = start_idx + self.sequence_length
            target_start = end_idx
            target_end = target_start + self.prediction_horizon
            
            X[i] = features[start_idx:end_idx]
            y[i] = targets[target_start:target_end]
        
        return X, y
    
    def create_single_sequence(self, features: np.ndarray) -> np.ndarray:
        """Create a single sequence for prediction (no target needed)."""
        if len(features) < self.sequence_length:
            raise ValueError(f"Not enough data points. Need {self.sequence_length}, got {len(features)}")
        
        # Take the last sequence_length rows
        sequence = features[-self.sequence_length:]
        return sequence.reshape(1, self.sequence_length, -1)

print("✅ Feature engineering functions defined")

## LSTM Model Architecture

In [ ]:
class LSTMForecaster(nn.Module):
    """
    LSTM-based forecasting model with attention mechanism.
    
    Features:
    - Multi-layer LSTM for sequence modeling
    - Multi-head attention for important pattern focus
    - Uncertainty quantification (mean + variance prediction)
    - Dropout for regularization
    """
    
    def __init__(self, config: ModelConfig):
        super(LSTMForecaster, self).__init__()
        self.config = config
        self.hidden_size = config.hidden_size
        self.num_layers = config.num_layers
        
        # LSTM layers
        self.lstm = nn.LSTM(
            input_size=config.input_features,
            hidden_size=config.hidden_size,
            num_layers=config.num_layers,
            dropout=config.dropout if config.num_layers > 1 else 0,
            batch_first=True,
            bidirectional=False
        )
        
        # Multi-head attention mechanism
        self.attention = nn.MultiheadAttention(
            embed_dim=config.hidden_size,
            num_heads=config.attention_heads,
            dropout=config.dropout,
            batch_first=True
        )
        
        # Layer normalization
        self.layer_norm = nn.LayerNorm(config.hidden_size)
        
        # Output layers
        self.dropout = nn.Dropout(config.dropout)
        
        # Separate heads for mean and variance prediction
        self.mean_head = nn.Sequential(
            nn.Linear(config.hidden_size, config.hidden_size // 2),
            nn.ReLU(),
            nn.Dropout(config.dropout),
            nn.Linear(config.hidden_size // 2, config.prediction_horizon)
        )
        
        self.variance_head = nn.Sequential(
            nn.Linear(config.hidden_size, config.hidden_size // 2),
            nn.ReLU(), 
            nn.Dropout(config.dropout),
            nn.Linear(config.hidden_size // 2, config.prediction_horizon),
            nn.Softplus()  # Ensure positive variance
        )
        
        # Initialize weights
        self._initialize_weights()
    
    def _initialize_weights(self):
        """Initialize model weights using Xavier/Glorot initialization."""
        for name, param in self.named_parameters():
            if 'weight_ih' in name:
                nn.init.xavier_uniform_(param.data)
            elif 'weight_hh' in name:
                nn.init.orthogonal_(param.data)
            elif 'bias' in name:
                param.data.fill_(0)
    
    def forward(self, x: torch.Tensor) -> Tuple[torch.Tensor, torch.Tensor]:
        """Forward pass through the model."""
        batch_size = x.size(0)
        
        # Initialize LSTM hidden states
        h0 = torch.zeros(self.num_layers, batch_size, self.hidden_size, device=x.device)
        c0 = torch.zeros(self.num_layers, batch_size, self.hidden_size, device=x.device)
        
        # LSTM forward pass
        lstm_out, _ = self.lstm(x, (h0, c0))
        
        # Apply attention mechanism
        attn_out, attention_weights = self.attention(lstm_out, lstm_out, lstm_out)
        
        # Residual connection and layer normalization
        lstm_out = self.layer_norm(lstm_out + attn_out)
        
        # Use the last timestep output
        last_output = lstm_out[:, -1, :]  # Shape: (batch_size, hidden_size)
        
        # Apply dropout
        features = self.dropout(last_output)
        
        # Predict mean and variance
        predicted_mean = self.mean_head(features)
        predicted_variance = self.variance_head(features)
        
        return predicted_mean, predicted_variance
    
    def predict_with_uncertainty(self, x: torch.Tensor, num_samples: int = 100) -> Tuple[torch.Tensor, torch.Tensor, torch.Tensor]:
        """Make predictions with uncertainty quantification using Monte Carlo dropout."""
        self.train()  # Enable dropout for MC sampling
        
        predictions = []
        variances = []
        
        with torch.no_grad():
            for _ in range(num_samples):
                pred_mean, pred_var = self.forward(x)
                predictions.append(pred_mean)
                variances.append(pred_var)
        
        predictions = torch.stack(predictions)  # (num_samples, batch_size, prediction_horizon)
        variances = torch.stack(variances)
        
        # Calculate uncertainties
        mean_prediction = predictions.mean(dim=0)
        epistemic_uncertainty = predictions.var(dim=0)  # Model uncertainty
        aleatoric_uncertainty = variances.mean(dim=0)   # Data uncertainty
        
        self.eval()  # Return to eval mode
        
        return mean_prediction, epistemic_uncertainty, aleatoric_uncertainty


class EarlyStopping:
    """Early stopping utility to prevent overfitting."""
    
    def __init__(self, patience: int = 10, min_delta: float = 1e-6):
        self.patience = patience
        self.min_delta = min_delta
        self.counter = 0
        self.best_loss = float('inf')
        self.early_stop = False
    
    def __call__(self, val_loss: float) -> bool:
        if val_loss < self.best_loss - self.min_delta:
            self.best_loss = val_loss
            self.counter = 0
        else:
            self.counter += 1
            if self.counter >= self.patience:
                self.early_stop = True
        
        return self.early_stop


def gaussian_nll_loss(y_true: torch.Tensor, y_pred_mean: torch.Tensor, y_pred_var: torch.Tensor) -> torch.Tensor:
    """Gaussian negative log-likelihood loss for uncertainty-aware training."""
    # Add small epsilon to prevent log(0)
    epsilon = 1e-8
    y_pred_var = y_pred_var + epsilon
    
    # Compute negative log-likelihood
    nll = 0.5 * (torch.log(y_pred_var) + (y_true - y_pred_mean)**2 / y_pred_var)
    return nll.mean()


class ModelEvaluator:
    """Utility class for model evaluation metrics."""
    
    @staticmethod
    def calculate_metrics(y_true: np.ndarray, y_pred: np.ndarray) -> dict:
        """Calculate comprehensive evaluation metrics."""
        # Handle multi-dimensional arrays by flattening if needed
        if y_true.ndim > 1:
            y_true = y_true.flatten()
        if y_pred.ndim > 1:
            y_pred = y_pred.flatten()
        
        # Basic error metrics
        errors = y_true - y_pred
        absolute_errors = np.abs(errors)
        squared_errors = errors ** 2
        
        mse = np.mean(squared_errors)
        mae = np.mean(absolute_errors)
        rmse = np.sqrt(mse)
        
        # Percentage-based metrics
        mape = np.mean(np.abs((y_true - y_pred) / (np.abs(y_true) + 1e-8))) * 100
        
        # Mean Absolute Scaled Error (MASE) - scaled by naive forecast
        naive_forecast_mae = np.mean(np.abs(np.diff(y_true)))
        mase = mae / (naive_forecast_mae + 1e-8)
        
        # Directional accuracy (did we predict the right direction?)
        if len(y_true) > 1:
            y_true_diff = np.diff(y_true)
            y_pred_diff = np.diff(y_pred)
            directional_accuracy = np.mean(np.sign(y_true_diff) == np.sign(y_pred_diff)) * 100
        else:
            directional_accuracy = 0.0
        
        # R-squared
        ss_res = np.sum(squared_errors)
        ss_tot = np.sum((y_true - np.mean(y_true)) ** 2)
        r2 = 1 - (ss_res / (ss_tot + 1e-8))
        
        # Additional metrics
        median_ae = np.median(absolute_errors)
        max_error = np.max(absolute_errors)
        std_error = np.std(errors)
        
        # Symmetric metrics
        smape = 200 * np.mean(np.abs(y_pred - y_true) / (np.abs(y_pred) + np.abs(y_true) + 1e-8))
        
        return {
            'mse': float(mse),
            'mae': float(mae),
            'rmse': float(rmse),
            'mape': float(mape),
            'smape': float(smape),
            'mase': float(mase),
            'r2': float(r2),
            'directional_accuracy': float(directional_accuracy),
            'median_absolute_error': float(median_ae),
            'max_error': float(max_error),
            'std_error': float(std_error),
            'mean_error': float(np.mean(errors))
        }

print("✅ LSTM model architecture defined")

## Training Functions

In [ ]:
class ModelTrainer:
    """Complete model training pipeline for FX forecasting."""
    
    def __init__(self, config: TrainingConfig):
        self.config = config
        self.device = self._get_device(config.device)
        self.model = None
        self.feature_engineer = None
        self.sequence_generator = None
        self.optimizer = None
        self.scheduler = None
        
        logger.info(f"Initialized ModelTrainer on device: {self.device}")
    
    def _get_device(self, device: str) -> torch.device:
        """Get appropriate device for training."""
        if device == "auto":
            return torch.device("cuda" if torch.cuda.is_available() else "cpu")
        return torch.device(device)
    
    def prepare_data(self, df: pd.DataFrame) -> Tuple[np.ndarray, np.ndarray]:
        """Prepare data for training."""
        logger.info(f"Preparing data: {len(df)} samples")
        
        # Feature engineering
        self.feature_engineer = FeatureEngineering(self.config.feature_config)
        df_features = self.feature_engineer.prepare_features(df)
        
        if df_features.empty:
            raise ValueError("Feature engineering resulted in empty dataset")
        
        # Get feature columns and fit scaler
        feature_columns = self.feature_engineer.get_feature_columns(df_features)
        logger.info(f"Feature columns ({len(feature_columns)}): {feature_columns[:10]}...")
        
        features_scaled = self.feature_engineer.fit_transform_features(df_features, feature_columns)
        
        # Update model config with actual number of features
        self.config.model_config.input_features = len(feature_columns)
        
        # Extract targets
        targets = df_features['rate'].values
        
        # Generate sequences
        self.sequence_generator = SequenceGenerator(
            self.config.model_config.sequence_length,
            self.config.model_config.prediction_horizon
        )
        
        X, y = self.sequence_generator.create_sequences(features_scaled, targets)
        
        logger.info(f"Generated sequences - X: {X.shape}, y: {y.shape}")
        
        return X, y
    
    def train_model(self, X: np.ndarray, y: np.ndarray) -> TrainingResult:
        """Train the LSTM model."""
        start_time = time.time()
        
        # Split data
        split_idx = int(len(X) * (1 - self.config.validation_split))
        X_train, X_val = X[:split_idx], X[split_idx:]
        y_train, y_val = y[:split_idx], y[split_idx:]
        
        logger.info(f"Training split: {len(X_train)} train, {len(X_val)} validation")
        
        # Convert to tensors
        X_train_tensor = torch.FloatTensor(X_train).to(self.device)
        y_train_tensor = torch.FloatTensor(y_train).to(self.device)
        X_val_tensor = torch.FloatTensor(X_val).to(self.device)
        y_val_tensor = torch.FloatTensor(y_val).to(self.device)
        
        # Create model
        self.model = LSTMForecaster(self.config.model_config).to(self.device)
        
        # Setup optimizer and scheduler
        self.optimizer = torch.optim.Adam(self.model.parameters(), lr=self.config.learning_rate)
        self.scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(
            self.optimizer, mode='min', factor=0.5, patience=5
        )
        
        # Training loop
        early_stopping = EarlyStopping(patience=self.config.patience)
        train_losses = []
        val_losses = []
        best_val_loss = float('inf')
        best_epoch = 0
        
        for epoch in range(self.config.max_epochs):
            # Training phase
            self.model.train()
            train_loss = 0.0
            
            # Create batches
            n_batches = len(X_train) // self.config.batch_size
            for i in range(n_batches):
                start_idx = i * self.config.batch_size
                end_idx = min(start_idx + self.config.batch_size, len(X_train))
                
                batch_X = X_train_tensor[start_idx:end_idx]
                batch_y = y_train_tensor[start_idx:end_idx]
                
                self.optimizer.zero_grad()
                
                # Forward pass
                pred_mean, pred_var = self.model(batch_X)
                
                # Compute loss (using Gaussian NLL)
                loss = gaussian_nll_loss(batch_y, pred_mean, pred_var)
                
                # Backward pass
                loss.backward()
                torch.nn.utils.clip_grad_norm_(self.model.parameters(), max_norm=1.0)
                self.optimizer.step()
                
                train_loss += loss.item()
            
            avg_train_loss = train_loss / n_batches
            train_losses.append(avg_train_loss)
            
            # Validation phase
            self.model.eval()
            val_loss = 0.0
            
            with torch.no_grad():
                pred_mean, pred_var = self.model(X_val_tensor)
                val_loss = gaussian_nll_loss(y_val_tensor, pred_mean, pred_var).item()
            
            val_losses.append(val_loss)
            
            # Learning rate scheduling
            self.scheduler.step(val_loss)
            
            # Save best model
            if val_loss < best_val_loss:
                best_val_loss = val_loss
                best_epoch = epoch
            
            # Print progress
            if epoch % 10 == 0 or epoch == self.config.max_epochs - 1:
                logger.info(f"Epoch {epoch:3d}: Train Loss: {avg_train_loss:.6f}, Val Loss: {val_loss:.6f}")
            
            # Early stopping
            if early_stopping(val_loss):
                logger.info(f"Early stopping at epoch {epoch}")
                break
        
        # Calculate final metrics
        self.model.eval()
        with torch.no_grad():
            val_pred_mean, _ = self.model(X_val_tensor)
            val_pred_np = val_pred_mean.cpu().numpy()
            val_true_np = y_val
            
            val_metrics = ModelEvaluator.calculate_metrics(val_true_np, val_pred_np)
        
        training_time = time.time() - start_time
        
        result = TrainingResult(
            train_losses=train_losses,
            val_losses=val_losses,
            val_metrics=val_metrics,
            training_time=training_time,
            best_epoch=best_epoch,
            converged=not early_stopping.early_stop
        )
        
        logger.info(f"Training completed in {training_time:.2f}s")
        logger.info(f"Best validation MAPE: {val_metrics['mape']:.2f}%")
        
        return result
    
    def save_model(self, path: str) -> None:
        """Save trained model to disk."""
        if self.model is None:
            raise ValueError("No model to save")
        
        save_dict = {
            'model_state_dict': self.model.state_dict(),
            'config': self.config,
            'feature_engineer': self.feature_engineer,
            'sequence_generator': self.sequence_generator,
        }
        
        with open(path, 'wb') as f:
            pickle.dump(save_dict, f)
        
        logger.info(f"Model saved to {path}")
    
    def load_model(self, path: str) -> None:
        """Load trained model from disk."""
        with open(path, 'rb') as f:
            save_dict = pickle.load(f)
        
        self.config = save_dict['config']
        self.feature_engineer = save_dict['feature_engineer']
        self.sequence_generator = save_dict['sequence_generator']
        
        # Create and load model
        self.model = LSTMForecaster(self.config.model_config).to(self.device)
        self.model.load_state_dict(save_dict['model_state_dict'])
        self.model.eval()
        
        logger.info(f"Model loaded from {path}")

print("✅ Training functions defined")

## 1. Data Collection and Exploration

Let's start by downloading historical FX data and exploring its characteristics.

In [ ]:
# Configuration for data collection
CURRENCY_PAIRS = ['USD/EUR', 'USD/GBP', 'EUR/GBP']
DATA_PERIOD = "1y"  # 1 year of historical data
DATA_INTERVAL = "1h"  # Hourly data

print("📊 Starting Data Collection")
print(f"Currency Pairs: {CURRENCY_PAIRS}")
print(f"Period: {DATA_PERIOD}, Interval: {DATA_INTERVAL}")
print("=" * 50)

# Initialize data collector
collector = YFinanceDataCollector()

# Check for existing data first
available_data = collector.get_available_data()
if available_data:
    print("📁 Found existing data:")
    for data_info in available_data:
        print(f"   {data_info['currency_pair']}: {data_info['records']} records "
              f"({data_info['start_date'].date()} to {data_info['end_date'].date()})")
    
    # Load existing data
    fx_data = {}
    for pair in CURRENCY_PAIRS:
        df = collector.load_historical_data(pair, DATA_PERIOD, DATA_INTERVAL)
        if df is not None:
            fx_data[pair] = df
else:
    # Download fresh data
    fx_data = collector.download_historical_data(
        currency_pairs=CURRENCY_PAIRS,
        period=DATA_PERIOD,
        interval=DATA_INTERVAL,
        save=True
    )

print(f"\n✅ Data collection complete! Loaded {len(fx_data)} currency pairs.")

In [ ]:
# Data exploration and visualization
fig, axes = plt.subplots(len(fx_data), 1, figsize=(15, 4 * len(fx_data)))
if len(fx_data) == 1:
    axes = [axes]

summaries = {}
for i, (pair, df) in enumerate(fx_data.items()):
    # Get summary statistics
    summary = collector.get_data_summary(df)
    summaries[pair] = summary
    
    # Print summary
    print(f"\n📈 {pair} Summary:")
    print(f"   Records: {summary['total_records']:,}")
    print(f"   Date Range: {summary['date_range']['start']} to {summary['date_range']['end']}")
    print(f"   Duration: {summary['date_range']['days']} days")
    print(f"   Rate Range: {summary['rate_statistics']['min']:.6f} - {summary['rate_statistics']['max']:.6f}")
    print(f"   Mean: {summary['rate_statistics']['mean']:.6f} ± {summary['rate_statistics']['std']:.6f}")
    print(f"   Missing Values: {summary['missing_values']}")
    print(f"   Avg Records/Day: {summary['data_quality']['avg_records_per_day']:.1f}")
    
    # Plot time series
    axes[i].plot(df['timestamp'], df['rate'], linewidth=0.8, alpha=0.8)
    axes[i].set_title(f'{pair} Exchange Rate Over Time', fontsize=14, fontweight='bold')
    axes[i].set_xlabel('Date')
    axes[i].set_ylabel('Exchange Rate')
    axes[i].grid(True, alpha=0.3)
    
    # Add statistics text box
    stats_text = f"Records: {summary['total_records']:,}\nMean: {summary['rate_statistics']['mean']:.6f}\nStd: {summary['rate_statistics']['std']:.6f}"
    axes[i].text(0.02, 0.98, stats_text, transform=axes[i].transAxes, 
                bbox=dict(boxstyle='round', facecolor='white', alpha=0.8),
                verticalalignment='top', fontsize=10)

plt.tight_layout()
plt.show()

## 2. Model Configuration and Training

Configure the LSTM model and training parameters. You can easily modify these to experiment with different configurations.

In [ ]:
# Model Configuration - Experiment with these parameters!
model_config = ModelConfig(
    sequence_length=168,  # 1 week of hourly data (7 * 24)
    prediction_horizon=24,  # Predict 24 hours ahead
    hidden_size=64,  # LSTM hidden size
    num_layers=2,  # Number of LSTM layers
    dropout=0.2,  # Dropout rate for regularization
    attention_heads=8  # Multi-head attention
)

# Feature Engineering Configuration
feature_config = FeatureConfig(
    rsi_period=14,  # RSI period
    ma_short_period=12,  # Short moving average
    ma_long_period=26,  # Long moving average
    bb_period=20,  # Bollinger Bands period
    volatility_windows=[6, 12, 24, 48],  # Volatility calculation windows
    lag_periods=[1, 2, 3, 6, 12, 24]  # Lagged features
)

# Training Configuration
training_config = TrainingConfig(
    model_config=model_config,
    feature_config=feature_config,
    max_epochs=50,  # Maximum training epochs
    batch_size=32,  # Batch size
    learning_rate=0.001,  # Learning rate
    patience=10,  # Early stopping patience
    validation_split=0.2  # Validation split ratio
)

print("🏗️  Model Configuration:")
print(f"   Sequence Length: {model_config.sequence_length} hours")
print(f"   Prediction Horizon: {model_config.prediction_horizon} hours")
print(f"   Hidden Size: {model_config.hidden_size}")
print(f"   Layers: {model_config.num_layers}")
print(f"   Dropout: {model_config.dropout}")
print(f"   Attention Heads: {model_config.attention_heads}")
print(f"   Max Epochs: {training_config.max_epochs}")
print(f"   Batch Size: {training_config.batch_size}")
print(f"   Learning Rate: {training_config.learning_rate}")

## 3. Train Models for Each Currency Pair

Now let's train LSTM models for each currency pair and evaluate their performance.

In [ ]:
def train_single_model(df: pd.DataFrame, currency_pair: str, config: TrainingConfig):
    """
    Train a single LSTM model for a currency pair.
    """
    print(f"\n🧠 Training ML Model for {currency_pair}")
    print("=" * 50)
    
    print(f"📊 Training data: {len(df)} records")
    print(f"   Date range: {df['timestamp'].min()} to {df['timestamp'].max()}")
    print(f"   Duration: {(df['timestamp'].max() - df['timestamp'].min()).days} days")
    
    # Initialize trainer
    trainer = ModelTrainer(config)
    
    print("🔄 Preparing training data...")
    X, y = trainer.prepare_data(df)
    
    print(f"   Training sequences: {X.shape[0]}")
    print(f"   Input shape: {X.shape}")
    print(f"   Target shape: {y.shape}")
    print(f"   Features per timestep: {X.shape[2]}")
    
    if X.shape[0] < 100:
        print("⚠️  Warning: Very few training sequences. Consider more data or shorter sequence length.")
    
    print("\n🚀 Starting model training...")
    training_result = trainer.train_model(X, y)
    
    print(f"\n✅ Training completed!")
    print(f"   Training time: {training_result.training_time:.2f} seconds")
    print(f"   Best epoch: {training_result.best_epoch}")
    print(f"   Converged: {training_result.converged}")
    print(f"   Final train loss: {training_result.train_losses[-1]:.6f}")
    print(f"   Final val loss: {training_result.val_losses[-1]:.6f}")
    
    return trainer, training_result

# Train models for each currency pair
trained_models = {}
training_results = {}

for pair, df in fx_data.items():
    if len(df) < 1000:  # Need sufficient data
        print(f"⚠️  Skipping {pair}: insufficient data ({len(df)} records)")
        continue
    
    trainer, result = train_single_model(df, pair, training_config)
    
    if trainer and result:
        trained_models[pair] = trainer
        training_results[pair] = result
        
        # Save trained model
        model_path = f"models/{pair.replace('/', '_')}_experiment.pkl"
        trainer.save_model(model_path)
        print(f"💾 Model saved to: {model_path}")
    else:
        print(f"❌ {pair} model training failed")

print(f"\n🎉 Training Summary: {len(trained_models)}/{len(fx_data)} models trained successfully")

## 4. Model Performance Analysis

Let's analyze the performance of our trained models with comprehensive metrics and visualizations.

In [ ]:
# Performance metrics and visualization
if trained_models:
    # Create performance comparison
    performance_data = []
    
    for pair, result in training_results.items():
        metrics = result.val_metrics
        performance_data.append({
            'Currency Pair': pair,
            'MSE': metrics['mse'],
            'MAE': metrics['mae'],
            'RMSE': metrics['rmse'],
            'MAPE (%)': metrics['mape'],
            'R²': metrics['r2'],
            'Directional Accuracy (%)': metrics['directional_accuracy'],
            'Training Time (s)': result.training_time
        })
        
        # Print detailed metrics
        print(f"\n📊 {pair} Performance Metrics:")
        print(f"   MSE (Mean Squared Error): {metrics['mse']:.8f}")
        print(f"   MAE (Mean Absolute Error): {metrics['mae']:.6f}")
        print(f"   RMSE (Root Mean Squared Error): {metrics['rmse']:.6f}")
        print(f"   MAPE (Mean Absolute Percentage Error): {metrics['mape']:.2f}%")
        print(f"   SMAPE (Symmetric MAPE): {metrics.get('smape', 'N/A'):.2f}%")
        print(f"   MASE (Mean Absolute Scaled Error): {metrics.get('mase', 'N/A'):.4f}")
        print(f"   R² (Coefficient of Determination): {metrics['r2']:.4f}")
        print(f"   Directional Accuracy: {metrics['directional_accuracy']:.2f}%")
    
    # Create comparison DataFrame
    performance_df = pd.DataFrame(performance_data)
    print("\n📋 Performance Comparison:")
    print(performance_df.round(6))
    
    # Plot training curves
    fig, axes = plt.subplots(2, len(trained_models), figsize=(5 * len(trained_models), 10))
    if len(trained_models) == 1:
        axes = axes.reshape(-1, 1)
    
    for i, (pair, result) in enumerate(training_results.items()):
        # Training and validation loss
        axes[0, i].plot(result.train_losses, label='Training Loss', alpha=0.8)
        axes[0, i].plot(result.val_losses, label='Validation Loss', alpha=0.8)
        axes[0, i].axvline(x=result.best_epoch, color='red', linestyle='--', alpha=0.5, label='Best Epoch')
        axes[0, i].set_title(f'{pair} - Training Curves')
        axes[0, i].set_xlabel('Epoch')
        axes[0, i].set_ylabel('Loss')
        axes[0, i].legend()
        axes[0, i].grid(True, alpha=0.3)
        
        # Metrics bar plot
        metrics_names = ['MAPE (%)', 'R²', 'Dir. Acc. (%)']
        metrics_values = [
            result.val_metrics['mape'],
            result.val_metrics['r2'] * 100,  # Scale R² for visibility
            result.val_metrics['directional_accuracy']
        ]
        
        bars = axes[1, i].bar(metrics_names, metrics_values, alpha=0.7)
        axes[1, i].set_title(f'{pair} - Key Metrics')
        axes[1, i].set_ylabel('Value')
        
        # Add value labels on bars
        for bar, value in zip(bars, metrics_values):
            height = bar.get_height()
            axes[1, i].text(bar.get_x() + bar.get_width()/2., height + height*0.01,
                           f'{value:.2f}', ha='center', va='bottom')
    
    plt.tight_layout()
    plt.show()

else:
    print("❌ No models trained successfully")

## 5. Make Predictions with Uncertainty

Test the models by making predictions on recent data with uncertainty quantification.

In [ ]:
def make_predictions_with_uncertainty(trainer, df: pd.DataFrame, currency_pair: str, num_samples: int = 100):
    """
    Make predictions with uncertainty quantification.
    """
    print(f"\n🔮 Testing Predictions for {currency_pair}")
    print("=" * 40)
    
    # Use last 7 days of data for prediction testing
    test_data = df.tail(7 * 24).copy()  # Last 7 days
    
    if len(test_data) < trainer.config.model_config.sequence_length:
        print("⚠️  Insufficient test data")
        return None
    
    # Prepare features for the test data
    df_features = trainer.feature_engineer.prepare_features(test_data)
    feature_columns = trainer.feature_engineer.get_feature_columns(df_features)
    features_scaled = trainer.feature_engineer.transform_features(df_features)
    
    # Create sequence for prediction
    sequence = trainer.sequence_generator.create_single_sequence(features_scaled)
    
    # Make prediction
    trainer.model.eval()
    with torch.no_grad():
        sequence_tensor = torch.FloatTensor(sequence).to(trainer.device)
        
        # Get standard prediction
        pred_mean, pred_var = trainer.model(sequence_tensor)
        
        # Get prediction with uncertainty
        pred_mean_mc, epistemic_unc, aleatoric_unc = trainer.model.predict_with_uncertainty(
            sequence_tensor, num_samples=num_samples
        )
    
    # Convert to numpy
    predictions = pred_mean.cpu().numpy().flatten()
    predictions_mc = pred_mean_mc.cpu().numpy().flatten()
    epistemic_uncertainty = epistemic_unc.cpu().numpy().flatten()
    aleatoric_uncertainty = aleatoric_unc.cpu().numpy().flatten()
    
    # Get current and recent rates
    current_rate = test_data['rate'].iloc[-1]
    recent_rates = test_data['rate'].tail(24).values  # Last 24 hours
    
    print(f"📈 Current Rate: {current_rate:.6f}")
    print(f"📊 Recent 24h Range: {recent_rates.min():.6f} - {recent_rates.max():.6f}")
    print(f"📊 Recent 24h Change: {((current_rate - recent_rates[0]) / recent_rates[0] * 100):+.2f}%")
    
    # Predictions at different horizons
    horizons = [0, 5, 11, 23]  # 1h, 6h, 12h, 24h
    horizon_names = ['1-hour', '6-hour', '12-hour', '24-hour']
    
    print(f"\n🔮 Predictions (next 24 hours):")
    prediction_data = []
    
    for h, name in zip(horizons, horizon_names):
        if h < len(predictions):
            total_unc = np.sqrt(epistemic_uncertainty[h] + aleatoric_uncertainty[h])
            change = ((predictions[h] - current_rate) / current_rate) * 100
            
            print(f"   {name} ahead: {predictions[h]:.6f} ±{total_unc:.6f} ({change:+.3f}%)")
            
            prediction_data.append({
                'horizon': name,
                'prediction': predictions[h],
                'uncertainty': total_unc,
                'change_pct': change
            })
    
    # Uncertainty analysis
    avg_epistemic = np.mean(epistemic_uncertainty[:24])
    avg_aleatoric = np.mean(aleatoric_uncertainty[:24])
    total_uncertainty = np.sqrt(avg_epistemic + avg_aleatoric)
    
    print(f"\n🎯 Uncertainty Analysis:")
    print(f"   Model Uncertainty (Epistemic): {avg_epistemic:.6f}")
    print(f"   Data Uncertainty (Aleatoric): {avg_aleatoric:.6f}")
    print(f"   Total Uncertainty: {total_uncertainty:.6f}")
    
    # Confidence assessment
    confidence = max(0.0, min(1.0, 1.0 - total_uncertainty))
    print(f"   Model Confidence: {confidence:.2f} ({confidence*100:.0f}%)")
    
    return {
        'predictions': predictions,
        'uncertainty_epistemic': epistemic_uncertainty,
        'uncertainty_aleatoric': aleatoric_uncertainty,
        'current_rate': current_rate,
        'prediction_data': prediction_data
    }

# Make predictions for each trained model
prediction_results = {}

for pair, trainer in trained_models.items():
    df = fx_data[pair]
    result = make_predictions_with_uncertainty(trainer, df, pair, num_samples=100)
    if result:
        prediction_results[pair] = result

print(f"\n✅ Predictions completed for {len(prediction_results)} models")

## 6. Prediction Visualization

Visualize the predictions with uncertainty bands.

In [ ]:
# Visualize predictions with uncertainty
if prediction_results:
    fig, axes = plt.subplots(len(prediction_results), 1, figsize=(15, 5 * len(prediction_results)))
    if len(prediction_results) == 1:
        axes = [axes]
    
    for i, (pair, result) in enumerate(prediction_results.items()):
        # Get recent data for context
        recent_data = fx_data[pair].tail(48)  # Last 48 hours
        
        # Plot historical data
        axes[i].plot(range(len(recent_data)), recent_data['rate'].values, 
                    label='Historical', color='blue', alpha=0.7, linewidth=1.5)
        
        # Plot predictions
        pred_start = len(recent_data)
        pred_hours = np.arange(pred_start, pred_start + len(result['predictions']))
        
        axes[i].plot(pred_hours, result['predictions'], 
                    label='Predictions', color='red', alpha=0.8, linewidth=2)
        
        # Plot uncertainty bands
        total_uncertainty = np.sqrt(result['uncertainty_epistemic'] + result['uncertainty_aleatoric'])
        
        axes[i].fill_between(pred_hours,
                            result['predictions'] - 1.96 * total_uncertainty,
                            result['predictions'] + 1.96 * total_uncertainty,
                            alpha=0.3, color='red', label='95% Confidence')
        
        # Add current rate line
        axes[i].axhline(y=result['current_rate'], color='green', linestyle='--', 
                       alpha=0.7, label=f'Current Rate: {result["current_rate"]:.6f}')
        
        axes[i].axvline(x=pred_start, color='black', linestyle=':', alpha=0.5, label='Prediction Start')
        
        axes[i].set_title(f'{pair} - 24-hour Predictions with Uncertainty', fontweight='bold')
        axes[i].set_xlabel('Hours')
        axes[i].set_ylabel('Exchange Rate')
        axes[i].legend()
        axes[i].grid(True, alpha=0.3)
        
        # Add prediction summary as text
        summary_text = f"24h Prediction: {result['predictions'][23]:.6f}\n"
        summary_text += f"Expected Change: {((result['predictions'][23] - result['current_rate']) / result['current_rate'] * 100):+.2f}%"
        
        axes[i].text(0.02, 0.98, summary_text, transform=axes[i].transAxes,
                    bbox=dict(boxstyle='round', facecolor='yellow', alpha=0.8),
                    verticalalignment='top', fontsize=10)
    
    plt.tight_layout()
    plt.show()

else:
    print("❌ No prediction results to visualize")

## 7. Model Comparison and Summary

Compare all models and summarize the experiment results.

In [ ]:
# Create comprehensive summary
if trained_models and prediction_results:
    print("🎉 EXPERIMENT SUMMARY")
    print("=" * 60)
    
    summary_data = []
    
    for pair in trained_models.keys():
        training_result = training_results[pair]
        prediction_result = prediction_results[pair]
        
        # Get 24-hour prediction
        pred_24h = prediction_result['predictions'][23] if len(prediction_result['predictions']) > 23 else prediction_result['predictions'][-1]
        change_24h = ((pred_24h - prediction_result['current_rate']) / prediction_result['current_rate']) * 100
        
        total_unc_24h = np.sqrt(
            prediction_result['uncertainty_epistemic'][23] + prediction_result['uncertainty_aleatoric'][23]
        ) if len(prediction_result['predictions']) > 23 else 0
        
        summary_data.append({
            'Currency Pair': pair,
            'MAPE (%)': f"{training_result.val_metrics['mape']:.2f}",
            'R²': f"{training_result.val_metrics['r2']:.4f}",
            'Dir. Accuracy (%)': f"{training_result.val_metrics['directional_accuracy']:.1f}",
            'Current Rate': f"{prediction_result['current_rate']:.6f}",
            '24h Prediction': f"{pred_24h:.6f}",
            'Expected Change (%)': f"{change_24h:+.2f}",
            'Uncertainty': f"±{total_unc_24h:.6f}",
            'Training Time (s)': f"{training_result.training_time:.1f}"
        })
    
    summary_df = pd.DataFrame(summary_data)
    print(summary_df.to_string(index=False))
    
    print(f"\n🏆 Best Performing Model (by MAPE):")
    best_pair = min(training_results.keys(), 
                   key=lambda k: training_results[k].val_metrics['mape'])
    best_mape = training_results[best_pair].val_metrics['mape']
    print(f"   {best_pair}: {best_mape:.2f}% MAPE")
    
    print(f"\n📊 Average Performance:")
    avg_mape = np.mean([result.val_metrics['mape'] for result in training_results.values()])
    avg_r2 = np.mean([result.val_metrics['r2'] for result in training_results.values()])
    avg_dir_acc = np.mean([result.val_metrics['directional_accuracy'] for result in training_results.values()])
    
    print(f"   MAPE: {avg_mape:.2f}%")
    print(f"   R²: {avg_r2:.4f}")
    print(f"   Directional Accuracy: {avg_dir_acc:.1f}%")
    
    print(f"\n💾 Trained models saved in 'models/' directory")
    print("🔗 Ready for integration with Decision Engine")

else:
    print("❌ Experiment incomplete - no models trained or predictions made")

## 8. Experiment with Different Configurations

This section allows you to quickly experiment with different model configurations.

In [ ]:
# Quick experimentation function
def quick_experiment(currency_pair: str, 
                    sequence_length: int = 168,
                    hidden_size: int = 64,
                    num_layers: int = 2,
                    max_epochs: int = 30):
    """
    Quick experiment with custom parameters on a single currency pair.
    """
    if currency_pair not in fx_data:
        print(f"❌ Currency pair {currency_pair} not available")
        return
    
    print(f"🧪 Quick Experiment: {currency_pair}")
    print(f"   Sequence Length: {sequence_length}")
    print(f"   Hidden Size: {hidden_size}")
    print(f"   Layers: {num_layers}")
    print(f"   Max Epochs: {max_epochs}")
    
    # Custom configuration
    custom_model_config = ModelConfig(
        sequence_length=sequence_length,
        prediction_horizon=24,
        hidden_size=hidden_size,
        num_layers=num_layers,
        dropout=0.2
    )
    
    custom_training_config = TrainingConfig(
        model_config=custom_model_config,
        feature_config=feature_config,
        max_epochs=max_epochs,
        batch_size=32,
        learning_rate=0.001,
        patience=5,  # Shorter patience for quick experiments
        validation_split=0.2
    )
    
    # Train model
    df = fx_data[currency_pair]
    trainer, result = train_single_model(df, currency_pair, custom_training_config)
    
    if trainer and result:
        print(f"\n📊 Quick Results:")
        print(f"   MAPE: {result.val_metrics['mape']:.2f}%")
        print(f"   R²: {result.val_metrics['r2']:.4f}")
        print(f"   Training Time: {result.training_time:.1f}s")
        
        # Quick prediction
        pred_result = make_predictions_with_uncertainty(trainer, df, currency_pair, num_samples=50)
        return trainer, result, pred_result
    
    return None

# Example: Experiment with different configurations
print("🔬 Try different configurations by running:")
print("quick_experiment('USD/EUR', sequence_length=96, hidden_size=32, max_epochs=20)")
print("quick_experiment('USD/GBP', sequence_length=240, hidden_size=128, num_layers=3)")

## 9. Model Persistence and Loading

Functions for saving and loading trained models for production use.

In [ ]:
# Model persistence utilities
def save_experiment_results(filename: str = "experiment_results.pkl"):
    """
    Save all experiment results for later analysis.
    """
    results = {
        'training_results': training_results,
        'prediction_results': prediction_results,
        'model_config': model_config,
        'feature_config': feature_config,
        'training_config': training_config,
        'fx_data_summary': {pair: collector.get_data_summary(df) for pair, df in fx_data.items()}
    }
    
    with open(filename, 'wb') as f:
        pickle.dump(results, f)
    
    print(f"💾 Experiment results saved to {filename}")

def load_trained_model(model_path: str, config: TrainingConfig):
    """
    Load a previously trained model for predictions.
    """
    trainer = ModelTrainer(config)
    trainer.load_model(model_path)
    print(f"📂 Model loaded from {model_path}")
    return trainer

# List available saved models
model_files = [f for f in os.listdir('models/') if f.endswith('.pkl')] if os.path.exists('models/') else []
print(f"💾 Available saved models ({len(model_files)}):")
for model_file in model_files:
    print(f"   - {model_file}")

# Save experiment results
if 'trained_models' in globals() and trained_models:
    save_experiment_results("latest_experiment_results.pkl")

## 10. Next Steps and Production Integration

Ideas for further experimentation and production deployment.

In [ ]:
print("🚀 NEXT STEPS FOR EXPERIMENTATION")
print("=" * 50)

print("\n1. 🔬 Model Architecture Experiments:")
print("   - Try different sequence lengths (96, 240, 336 hours)")
print("   - Experiment with attention mechanisms")
print("   - Test different LSTM hidden sizes (32, 128, 256)")
print("   - Add more layers or reduce for simpler models")

print("\n2. 📊 Feature Engineering Experiments:")
print("   - Add more technical indicators (MACD, Stochastic)")
print("   - Include economic calendar features")
print("   - Experiment with different volatility windows")
print("   - Test cross-currency correlation features")

print("\n3. 📈 Data Experiments:")
print("   - Use different time periods (2y, 5y historical data)")
print("   - Try different intervals (30m, 4h, daily)")
print("   - Include more currency pairs")
print("   - Add external data (economic indicators, news sentiment)")

print("\n4. 🎯 Training Experiments:")
print("   - Different loss functions (Huber, Focal Loss)")
print("   - Learning rate scheduling strategies")
print("   - Different optimizers (AdamW, RMSprop)")
print("   - Regularization techniques (weight decay, label smoothing)")

print("\n5. 🔮 Prediction Experiments:")
print("   - Multi-horizon predictions (1h, 6h, 24h, 7d)")
print("   - Ensemble methods (multiple model voting)")
print("   - Online learning and model adaptation")
print("   - Backtesting on historical data")

print("\n6. 🏭 Production Integration:")
print("   - Model serving with FastAPI")
print("   - Real-time data streaming")
print("   - Model monitoring and drift detection")
print("   - A/B testing framework")

print("\n" + "=" * 50)
print("Ready for experimentation! Modify the cells above to test different configurations.")
print("\n💡 Pro tip: Start with the 'quick_experiment' function for rapid testing!")
print("\n🧪 Example experiments to try:")
print("   quick_experiment('USD/EUR', sequence_length=96, hidden_size=32, max_epochs=20)")
print("   quick_experiment('EUR/GBP', sequence_length=240, hidden_size=128, num_layers=3, max_epochs=40)")